 This script provides a basis for the Morris screening algorithm. Please add code where appropriate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# STUDENTS: Define the function f
def f(x):
    # Placeholder for the cubic function; replace with actual implementation
    return x**3

# STUDENTS: Define upper and lower bounds (students need to provide values)
UB = np.array([])  # Upper bounds
LB = np.array([])  # Lower bounds
num_par = len(UB)

# Initialize algorithm
parameter_names = ['$q_1$', '$q_2$', '$q_3$', '$q_4$']
R = 200           # Number of samples we want
p = num_par       # Number of parameters
l = 60            # Number of levels
delta = l / (2 * (l - 1))  # Step Size
upper = UB
lower = LB
d = np.zeros((R, p))  # Store the elementary effects

# Use the randomization algorithm
A = np.zeros((p + 1, p))
for i in range(p):
    A[i + 1 : p + 1, i] = 1

qstar = np.random.uniform(0, 1, (R, p))
Jp = np.ones((p + 1, p))
J1 = np.ones((p + 1, 1))
P = np.eye(p)
UL_MAT = np.eye(p) * (upper - lower)

F_storage = [[] for _ in range(p + 1)]
for i in range(R):
    qcurr = qstar[i, :]
    pm1 = np.random.random(p)
    Dstar = np.diag((pm1 > 0.5).astype(int)) - np.diag((pm1 <= 0.5).astype(int))
    Pstar = P[np.argsort(np.random.random(p)), :]
    Astar = J1 @ qcurr.reshape(1, -1) + (delta / 2) * ((2 * A - Jp) @ Dstar + Jp) @ Pstar
    C = J1 * lower + Astar @ UL_MAT
    fpast = f(C[0, :])
    F_storage[0].append(fpast)
    for j in range(p):
        fstep = f(C[j + 1, :])
        par_sign = np.sign(C[j + 1, j] - C[j, j])
        d[i, j] = par_sign * (fstep - fpast) / delta
        fpast = fstep
        F_storage[j + 1].append(fpast)

mu = np.mean(d, axis=0)
mu_star = np.mean(np.abs(d), axis=0)
sigma = np.sqrt(np.sum((d - mu) ** 2, axis=0) / (R - 1))

# Plotting results
plt.figure()
for i in range(num_par):
    plt.plot(mu_star[i], sigma[i], 'k.', markersize=20)
    plt.text(mu_star[i] * 1.01, sigma[i], parameter_names[i], fontsize=16)
plt.xlabel('$\\mu^*$', fontsize=14)
plt.ylabel('$\\sigma$', fontsize=14)
plt.title('Morris Indices')
plt.show()

rank = np.sqrt(mu_star**2 + sigma**2)
plt.figure()
plt.bar(range(len(rank)), rank, tick_label=parameter_names)
plt.title('Parameter Rankings')
plt.show()
